# Retrieval-Augmented Generation (RAG)

In [1]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [2]:
# We populated the RAG with the data from the data/calories.csv file in
# the rag_setup.ipynb notebook
chroma_client = chromadb.PersistentClient("../chroma")
nutrition_db = chroma_client.get_collection("nutrition_db")


In [3]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

/home/vscode/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 47.8MiB/s]


[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
        Nutritional Information:
        - Calories: 89 per 100g
        - Energy: 374 kJ per 100g
        - Serving size reference: 100g

        This is a fruits food item that provides 89 calories per 100 grams.


[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana_juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fruit)Juices
        Nutritional Information:
        - Calories: 50 per 100g
        - Energy: 210 kJ per 100g
        - Serving size reference: 100ml

        This is a (fruit)juices food item that provides 50 calories per 100 grams.




In [11]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [ ]:
# calorie_lookup_tool('bananas')

'Nutrition Information:\nBanana (Fruits): 89.0 calories per 100g\nBanana Juice ((Fruit)Juices): 50.0 calories per 100g\nBanana Nut Bread (Pastries,Breads&Rolls): 326.0 calories per 100g'

In [12]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    """,
    tools = [calorie_lookup_tool],
)

In [13]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result.final_output)

- Per 100 g: banana 89 kcal; apple 52 kcal.
- Estimated total for one medium banana (~118 g) and one medium apple (~182 g): about 200 kcal.


In [14]:
question_db = chroma_client.get_collection("nutrition_qna")

In [20]:
results = question_db.query(query_texts=["i am pregnant. sugar"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

[('is_pregnancy', True)]
Question: How can one manage early signs of pregnancy such as nausea and bouts of vomiting?
        Answer: Frequent consumption of snacks has proven effective for many pregnant individuals in both preventing and overcoming morning sickness, which is often linked to fluctuating blood sugar levels.

        This Q&A pair provides information about nutrition and health topics.


[('is_pregnancy', True)]
Question: How does one deal with early pregnancy symptoms such as vomiting and dizziness?
        Answer: Eating snacks at regular intervals may assist in preventing or alleviating morning sickness during the initial stages of gestation by stabilizing blood sugar levels.

        This Q&A pair provides information about nutrition and health topics.




In [ ]:
@function_tool
def nutrition_qa(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up the answer to common nutrition-related questions.

    Args:
        query: A question related to proper nutrition or the lack of proper nutrition. Often the question pertains to a pregnant woman.

        max_results: The maximum number of results to return.

    Returns:
        A string containing the answer to the nutrition-related question.
    """

    results = question_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)

    return "Related answers to your question:\n" + "\n".join(formatted_results)

In [ ]:
#print(nutrition_qa("What is the correct amount of sugar to consume daily?"))

Related answers to your question:
Question: What daily amount of retinol or beta-carotene should an adult female consume?
        Answer: An adult woman should aim for a daily intake of 600 mcg of retinol or 2400 mcg of beta-carotene to meet her vitamin A needs. This can be obtained from various sources like leafy greens, fruits, and dairy products.

        This Q&A pair provides information about nutrition and health topics.
Question: Which kinds of sugar should be consumed in moderation for people with diabetes mellitus?
        Answer: People with diabetes mellitus need to limit their intake of simple sugars like monosaccharides and disaccharides.

        This Q&A pair provides information about nutrition and health topics.
Question: What common items provide most of our sugar intake?
        Answer: The primary sources of sugars that we consume in our diet include sweet treats like chocolates, candies, milk, fruit-based products, and also fruits themselves. Sugars can be naturall

In [24]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information
    You are also capable of answering questions about proper nutrition for human beings and also discussing malnutrition.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    If you need information related to nutrition or malnutrition always first use the nutrition_qa tool for helpful advice.
    """,
    tools = [calorie_lookup_tool, nutrition_qa],
)

In [28]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many calories does a serving of salmon have?",
    )
    print(result.final_output)

About 206 calories per 100 g serving of salmon (raw). A 3 oz (85 g) serving ≈ 175 kcal.
